In [1]:
import torch
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
# >>> a photography of a woman and her dog

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a woman and her dog on the beach
woman sitting on the beach with her dog and a cell phone


In [25]:
def generate_caption(img_name):
  img_root_dir = '/content/unzipped_images/train2017/train2017/'
  raw_image = Image.open(img_root_dir+img_name).convert('RGB')
  inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
  out = model.generate(**inputs)
  print(processor.decode(out[0], skip_special_tokens=True))
  return processor.decode(out[0], skip_special_tokens=True)

In [3]:
import zipfile
print("Unzipping main image files")
#zip_file_path is whereever the train2017.zip file is stored at, this path would then be used to downlod the zip file into local disk for faster retrieval
zip_file_path = '/content/drive/MyDrive/ShareGPT4V/data/coco/train2017.zip'
#zip_file_path = '/data2/wuyinjun/mscoco/train2017.zip'
#extract_dir holds the directory path where the extracted files will be stored
extract_dir = '/content/unzipped_images/train2017'
# extract_dir = 'C:/Users/YourUsername/Documents/unzipped_images/train2017'
#extract_dir = "/data2/wuyinjun/mscoco/images/"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)
print("Finished unzipping main image files")

Unzipping main image files
Finished unzipping main image files


In [4]:
import pickle
img_caption_file_name = '/content/drive/MyDrive/mscoco_40kdecomposed_dependencies_wholeexp.pkl'

with open(img_caption_file_name, 'rb') as f:
  df_mscoco_40k = pickle.load(f)

In [5]:
df_mscoco_40k

,id,image,caption_sharegpt4v,caption_mscoco,embedding,caption_triples_ls,groups,elapsed_time,num_groups
0,9,000000000009.jpg,"In the center of the image, a vibrant blue lun...",Closeup of bins of food that include broccoli ...,"[0.196618, -0.1124176, -0.16870016, 0.03273408...","""Closeup of bins of food.""|""Bins of food inclu...","0,1,2",12.546637,1
1,25,000000000025.jpg,This image captures a serene moment in a zoo e...,A giraffe eating food from the top of the tree.,"[0.3378602, 0.00365863, -0.32199287, 0.1653291...","""A giraffe.""|""Eating food from the top of the ...",0 | 1,9.056211,2
2,30,000000000030.jpg,"The image presents a serene garden scene, cent...",A flower vase is sitting on a porch stand.,"[-0.034444228, 0.29082185, -0.022862792, -0.42...","""A flower vase.""|""Sitting on a porch stand.""",0 | 1,9.195682,2
3,34,000000000034.jpg,This is a detailed description of the image:\n...,A zebra grazing on lush green grass in a field.,"[-0.12711053, -0.30761, 0.0056521297, -0.01875...","""A zebra grazing."" | ""Lush green grass in a fi...",0 | 1,8.578875,2
4,36,000000000036.jpg,"In the image, there is a woman standing in fro...",Woman in swim suit holding parasol on sunny day.,"[0.2885676, -0.18000656, -0.5650209, 0.0151512...","""Woman in swim suit."" | ""Woman holding parasol...","0,1,2",14.776144,1
...,...,...,...,...,...,...,...,...,...
43815,343235,000000343235.jpg,"In the given image, a tranquil scene unfolds i...",Three cats sitting looking out sliding glass d...,"[0.46538836, -0.31321126, -0.32928455, -0.3109...","""Three cats sitting looking out sliding glass ...","0,1",4.688795,1
43816,271681,000000271681.jpg,"In the heart of a bustling city, a young woman...",A woman holding a quiche in her hand and smiling,"[-0.022412345, 0.32375985, -0.32947338, -0.019...","""A woman.""|""A woman holding a quiche in her ha...","0,1,2",8.097479,1
43817,268726,000000268726.jpg,This is a black and white photograph capturing...,A large dog sleeping on top of a couch.,"[0.3253008, 0.43674475, -0.7003968, 0.13196589...","""A large dog.""|""Sleeping on top of a couch.""",0 | 1,4.469425,2
43818,478857,000000478857.jpg,The image captures a serene suburban scene. Do...,Stop signs on the side of the street near vehi...,"[0.39890462, -0.24824393, 0.27765816, -0.44707...","""Stop signs on the side of the street.""|""Near ...","0,1,2",7.731356,1


In [22]:
df_test = df_mscoco_40k

In [26]:
import time
start_time = time.time()
df_test['blip_caption'] = df_test['image'].apply(generate_caption)
print("--- %s seconds ---" % (time.time() - start_time))
pickle_file_path = '/content/drive/MyDrive/df_mscoco_40k_blip_captions_whole.pkl'
with open(pickle_file_path, 'wb') as f:
  pickle.dump(df_test, f)

Streaming output truncated to the last 5000 lines.
there is a table with a blue table cloth and a vase of flowers
there is a kitchen with white cabinets and a black floor
there is a paper plate with orange slices on it
surfers on the beach with their boards in hand and a man in the water
there is a man sitting on a couch in a living room
herd of cows grazing in a field with a house in the background
there are many cows that are standing in a field together
arafed baseball player holding a bat on a baseball field
several young men are standing around talking and having a conversation
there is a toilet in a stall with a bucket and a trash can
there are many people that are taking pictures of themselves on their cell phones
there is a painting of a basket of fruit and vegetables
there are three girls sitting on a bench eating donuts
there is a wooden cabinet with a basket on top of it
arafed man in a red robe and chainmail holding a cell phone
there is a couch and a chair in a living room

In [27]:
df_test['blip_caption_processed'] = df_test['blip_caption'].str.replace(r'^(there is|there are|image of|photo of)\s+', '', regex=True)
df_test['blip_caption_processed'] = df_test['blip_caption_processed'].str.replace(r'^\b(araf\w*)\b\s*', '', regex=True)

In [28]:
pickle_file_path = '/content/drive/MyDrive/df_mscoco_40k_blip_captions_processed.pkl'
with open(pickle_file_path, 'wb') as f:
  pickle.dump(df_test, f)

In [29]:
img_caption_file_name = '/content/drive/MyDrive/df_mscoco_40k_blip_captions_processed.pkl'
with open(img_caption_file_name, 'rb') as f:
  test = pickle.load(f)

In [30]:
test

,id,image,caption_sharegpt4v,caption_mscoco,embedding,caption_triples_ls,groups,elapsed_time,num_groups,blip_caption,blip_caption_processed
0,9,000000000009.jpg,"In the center of the image, a vibrant blue lun...",Closeup of bins of food that include broccoli ...,"[0.196618, -0.1124176, -0.16870016, 0.03273408...","""Closeup of bins of food.""|""Bins of food inclu...","0,1,2",12.546637,1,there are three trays of food that include bro...,"three trays of food that include broccoli, bre..."
1,25,000000000025.jpg,This image captures a serene moment in a zoo e...,A giraffe eating food from the top of the tree.,"[0.3378602, 0.00365863, -0.32199287, 0.1653291...","""A giraffe.""|""Eating food from the top of the ...",0 | 1,9.056211,2,there are two giraffes standing in a field nex...,two giraffes standing in a field next to a tree
2,30,000000000030.jpg,"The image presents a serene garden scene, cent...",A flower vase is sitting on a porch stand.,"[-0.034444228, 0.29082185, -0.022862792, -0.42...","""A flower vase.""|""Sitting on a porch stand.""",0 | 1,9.195682,2,there is a white vase with flowers on a ledge ...,a white vase with flowers on a ledge outside
3,34,000000000034.jpg,This is a detailed description of the image:\n...,A zebra grazing on lush green grass in a field.,"[-0.12711053, -0.30761, 0.0056521297, -0.01875...","""A zebra grazing."" | ""Lush green grass in a fi...",0 | 1,8.578875,2,zebra grazing in a field of grass with its hea...,zebra grazing in a field of grass with its hea...
4,36,000000000036.jpg,"In the image, there is a woman standing in fro...",Woman in swim suit holding parasol on sunny day.,"[0.2885676, -0.18000656, -0.5650209, 0.0151512...","""Woman in swim suit."" | ""Woman holding parasol...","0,1,2",14.776144,1,woman in a bathing suit holding a pink umbrell...,woman in a bathing suit holding a pink umbrell...
...,...,...,...,...,...,...,...,...,...,...,...
43815,343235,000000343235.jpg,"In the given image, a tranquil scene unfolds i...",Three cats sitting looking out sliding glass d...,"[0.46538836, -0.31321126, -0.32928455, -0.3109...","""Three cats sitting looking out sliding glass ...","0,1",4.688795,1,there are three cats sitting on the floor look...,three cats sitting on the floor looking out th...
43816,271681,000000271681.jpg,"In the heart of a bustling city, a young woman...",A woman holding a quiche in her hand and smiling,"[-0.022412345, 0.32375985, -0.32947338, -0.019...","""A woman.""|""A woman holding a quiche in her ha...","0,1,2",8.097479,1,smiling woman holding a slice of pizza on a ci...,smiling woman holding a slice of pizza on a ci...
43817,268726,000000268726.jpg,This is a black and white photograph capturing...,A large dog sleeping on top of a couch.,"[0.3253008, 0.43674475, -0.7003968, 0.13196589...","""A large dog.""|""Sleeping on top of a couch.""",0 | 1,4.469425,2,there is a dog that is laying on a couch with ...,a dog that is laying on a couch with pillows
43818,478857,000000478857.jpg,The image captures a serene suburban scene. Do...,Stop signs on the side of the street near vehi...,"[0.39890462, -0.24824393, 0.27765816, -0.44707...","""Stop signs on the side of the street.""|""Near ...","0,1,2",7.731356,1,there are two stop signs on the street in fron...,two stop signs on the street in front of a city
